In [ ]:
!git clone https://github.com/carlos-vinicios/Neural-Networks

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from sklearn import metrics
import numpy as np
import itertools
import matplotlib.pyplot as plt

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, 
                          title="Matriz de Confusão", 
                          cmap=plt.cm.Blues):
  plt.rcParams["axes.grid"] = False #esconde as linhas do grid
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)
  
  
  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalizada")
  else:
    print("Sem normalização")
   
  print(cm)
  
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], 
             horizontalalignment="center", 
             color="white" if cm[i, j] > thresh else "black")
  
  plt.tight_layout()
  plt.ylabel("Classes reais")
  plt.xlabel("Classes previstas")

In [ ]:
train_path = "Neural-Networks/train_base_over/train"
valid_path = "Neural-Networks/train_base_over/valid"
test_path = "Neural-Networks/train_base_over/test"


In [ ]:
model_vgg16 = VGG16()

#necessario criar um novo modelo sequencial para após aproveitar as camadas da VGG16, adicionando uma a uma
model = Sequential()
for layer in model_vgg16.layers:
    model.add(layer)

model.layers.pop() # retira a última dense layer contendo as 1000 classes

for layer in model.layers: #congela as camadas iniciais do modelo para que não aprendam novas caracteristicas
    layer.trainable = False

model.add(Dense(2, activation='softmax'))#adicionamos uma nova dense layer como softmax, para classificação entre melanoma e não melanoma

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit_generator(train_generator, steps_per_epoch=12, 
                    validation_data=validation_generator, 
                    validation_steps=4, 
                    epochs=6)

model.evaluate_generator(generator=validation_generator, steps=4)

In [ ]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224), 
        batch_size=batch_size,
        classes=["melanoma", "normais"])

validation_generator = test_datagen.flow_from_directory(
        valid_path,
        target_size=(224, 224),
        batch_size=batch_size,
        classes=["melanoma", "normais"])

test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(224, 224),
        batch_size=batch_size,
        classes=["melanoma", "normais"])


In [ ]:
teste_img, test_labels = next(test_generator)

test_labels = test_labels[:,0]

predict = model.predict_generator(test_generator, steps=1, verbose=0)

print(metrics.accuracy_score(test_labels, np.round(predict[:,0])))

print(metrics.classification_report(test_labels, np.round(predict[:,0])))

confusion = metrics.confusion_matrix(test_labels, np.round(predict[:,0]))
print()
print()
cm_plot_labels = ["melanomas", "normais"]

plot_confusion_matrix(confusion, cm_plot_labels, title="Matriz de Confusão")


In [ ]:
model.save("test1.h5")
print("Saved model to disk")